In [186]:
#Cargamos libreria
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special 
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [187]:
#Cargamos el dataset
df = pd.read_csv('Datos_limpios_mexico.csv')
df.head(5)

,Unnamed: 0,listing_url,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_verifications,neighbourhood_cleansed,property_type,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,0,https://www.airbnb.com/rooms/35797,2010-06-28,a few days or more,0%,92%,f,"['email', 'phone', 'work_email']",Cuajimalpa de Morelos,Entire villa,...,4.84,4.80,4.80,4.80,4.70,1.0,1.0,0.0,0.0,1.70
1,1,https://www.airbnb.com/rooms/44616,2010-08-09,within an hour,100%,93%,t,"['email', 'phone', 'work_email']",Cuauhtémoc,Entire home,...,4.70,4.87,4.78,4.98,4.48,8.0,4.0,1.0,0.0,0.41
2,2,https://www.airbnb.com/rooms/67703,2011-01-04,within an hour,71%,57%,t,"['email', 'phone']",Cuauhtémoc,Entire rental unit,...,4.75,4.94,4.92,4.98,4.91,2.0,2.0,0.0,0.0,0.31
3,3,https://www.airbnb.com/rooms/70644,2010-08-24,within a few hours,100%,100%,t,"['email', 'phone', 'work_email']",Coyoacán,Entire rental unit,...,4.96,4.96,4.98,4.96,4.92,3.0,2.0,1.0,0.0,0.83
4,4,https://www.airbnb.com/rooms/107078,2011-04-27,within an hour,100%,100%,t,"['email', 'phone']",Miguel Hidalgo,Entire loft,...,5.00,5.00,4.73,4.91,4.82,3.0,3.0,0.0,0.0,0.11


In [188]:
#Eliminamos las columnas que son innecesarias  
df = df.drop(['Unnamed: 0'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26582 entries, 0 to 26581
Data columns (total 50 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   listing_url                                   26582 non-null  object 
 1   host_since                                    26582 non-null  object 
 2   host_response_time                            26582 non-null  object 
 3   host_response_rate                            26582 non-null  object 
 4   host_acceptance_rate                          26582 non-null  object 
 5   host_is_superhost                             26582 non-null  object 
 6   host_verifications                            26582 non-null  object 
 7   neighbourhood_cleansed                        26582 non-null  object 
 8   property_type                                 26582 non-null  object 
 9   room_type                                     26582 non-null 

In [189]:
# Convertimos las columnas 'host_response_rate' y 'host_acceptance_rate' a tipo float
df['host_response_rate'] = df['host_response_rate'].str.rstrip('%').astype('float') / 100.0
df['host_acceptance_rate'] = df['host_acceptance_rate'].str.rstrip('%').astype('float') / 100.0


In [190]:

#Reemplazar los valores de true por 1 y false a 0 posteriormente lo verificamos 
df['host_is_superhost'] = df['host_is_superhost'].map({'t': True, 'f': False}).astype(int)

**Caso 1: host_is_superhost**

In [191]:
#Declaramos las variables dependientes e independientes para la regresion lodistica
var_ind = df[['review_scores_cleanliness', 'host_response_rate', 'host_acceptance_rate']]
var_dep = df['host_is_superhost']

In [192]:
#Redifinimos las variables
X = var_ind
y = var_dep

In [193]:
#Dividimos el conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [194]:
#Se escalan todos los datos
escalar = StandardScaler()

In [195]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, se utiliza el método fit_transform
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [196]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [197]:
#Entrenamos el algoritmo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [198]:
y_pred = algoritmo.predict(X_test)
y_pred

array([1, 0, 0, ..., 1, 1, 1])

In [199]:
#Verificamos la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de confusión:')
print(matriz)

Matriz de confusión:
[[3235 1378]
 [1289 2073]]


In [200]:
#Verificamos la precisión
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred, average="binary", pos_label=1)
print('Precisión: del modelo', precision)

Precisión: del modelo 0.6006954505940307


In [201]:
#calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo:', exactitud)

Exactitud del modelo: 0.6655799373040753


In [202]:
#Calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label=0)
print('Sensibilidad del modelo:', sensibilidad)

Sensibilidad del modelo: 0.7012789941469759


Se realiza un balance para obtener datos más equilibrados 

In [203]:
#Hacemos un conteo de los t y los f que exiten en host_is_superhost
df['host_is_superhost'].value_counts()

host_is_superhost
0    15346
1    11236
Name: count, dtype: int64

In [204]:
#Balanceamos el modelo 
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(class_weight='balanced')


In [205]:
#Entrenamos el modelo
model.fit(X_train, y_train)

LogisticRegression(class_weight='balanced')

In [206]:
#Realizamos la predicción con el modelo balanceado
y_pred_balanced = model.predict(X_test)
y_pred_ajustado = [1 if prob > 0.4 else 0 for prob in y_pred_balanced]


In [207]:
#Verificamos la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz_ajustada = confusion_matrix(y_test, y_pred_ajustado)
print('Matriz de confusión ajustada:')
print(matriz_ajustada)

Matriz de confusión ajustada:
[[2496 2117]
 [ 659 2703]]


In [208]:
#Realizamos la predicción con el modelo balanceado
from sklearn.metrics import precision_score
precision_ajustada = precision_score(y_test, y_pred_ajustado, average="binary", pos_label=1)
print('Precisión ajustada:', precision_ajustada)

Precisión ajustada: 0.5607883817427386


In [209]:
#Verificamos la exactitud y sensibilidad del modelo ajustado
from sklearn.metrics import accuracy_score, recall_score
exactitud_ajustada = accuracy_score(y_test, y_pred_ajustado)
print('Exactitud ajustada:', exactitud_ajustada)
sensibilidad_ajustada = recall_score(y_test, y_pred_ajustado, average="binary", pos_label=0)
print('Sensibilidad ajustada:', sensibilidad_ajustada)


Exactitud ajustada: 0.6519122257053291
Sensibilidad ajustada: 0.5410795577715153


**Caso 2: host_response_rate**

In [210]:
print(df['host_response_rate'].unique())  # Ver valores únicos


[0.   1.   0.71 0.9  0.66 0.83 0.91 0.88 0.5  0.89 0.99 0.67 0.33 0.2
 0.93 0.97 0.8  0.85 0.79 0.75 0.94 0.6  0.4  0.96 0.43 0.55 0.1  0.98
 0.73 0.7  0.86 0.11 0.95 0.92 0.81 0.25 0.57 0.78 0.62 0.76 0.82 0.14
 0.58 0.29 0.23 0.3  0.13 0.87 0.19 0.47 0.04 0.63 0.17 0.74 0.48 0.56
 0.64 0.49 0.09 0.54 0.38 0.77 0.36 0.61 0.39 0.69 0.16 0.27 0.72 0.53
 0.84 0.46 0.22 0.08 0.18 0.05]


In [211]:

# df['host_response_rate_category'] = df['host_response_rate'].apply(lambda x: 'Alta' if x >= 0.90 else 'Baja')
median_value = df['host_response_rate'].median()  # Calcula la mediana
df['host_response_rate_category'] = df['host_response_rate'].apply(
    lambda x: 'Alta' if x >= median_value else 'Baja'
)


In [212]:

# Verifica la nueva proporción
print(df['host_response_rate_category'].value_counts(normalize=True))


host_response_rate_category
Alta    0.79772
Baja    0.20228
Name: proportion, dtype: float64


In [213]:

print(df['host_response_rate_category'].value_counts(normalize=True))  # Ver proporciones


host_response_rate_category
Alta    0.79772
Baja    0.20228
Name: proportion, dtype: float64


In [214]:

#Declaramos las variables dependientes e independientes para la regresion lodistica
var_ind2 = df[['host_acceptance_rate', 'calculated_host_listings_count', 'review_scores_communication']]
var_dep2 = df['host_response_rate_category']


In [215]:

#Redifinimos las variables
X = var_ind2
y = var_dep2


In [216]:

#Dividimos el conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)


In [217]:

#Se escalan todos los datos
escalar2 = StandardScaler()


In [218]:

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, se utiliza el método fit_transform
X_train = escalar2.fit_transform(X_train)
X_test = escalar2.transform(X_test)


In [219]:

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo2 = LogisticRegression()


In [220]:

#Entrenamos el algoritmo
algoritmo2.fit(X_train, y_train)


LogisticRegression()

In [221]:

y_pred2 = algoritmo2.predict(X_test)
y_pred2


array(['Alta', 'Alta', 'Alta', ..., 'Alta', 'Alta', 'Alta'], dtype=object)

In [222]:

#Verificamos la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred2)
print('Matriz de confusión:')
print(matriz)


Matriz de confusión:
[[6226  200]
 [1293  256]]


In [223]:

#Verificamos la precisión
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred2, average="binary", pos_label="Alta")
print('Precisión: del modelo', precision)


Precisión: del modelo 0.828035643037638


In [224]:

#calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred2)
print('Exactitud del modelo:', exactitud)


Exactitud del modelo: 0.8127899686520376


In [225]:

#Calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred2, average="binary", pos_label="Alta")
print('Sensibilidad del modelo:', sensibilidad)

Sensibilidad del modelo: 0.9688764394646747


**Caso 3: host_acceptance_rate**

In [226]:
print(df['host_acceptance_rate'].unique())  # Ver valores únicos


[0.92 0.93 0.57 1.   0.98 0.78 0.95 0.67 0.75 0.88 0.   0.96 0.97 0.82
 0.8  0.99 0.81 0.61 0.5  0.86 0.44 0.94 0.25 0.3  0.6  0.43 0.62 0.89
 0.73 0.71 0.9  0.65 0.53 0.77 0.84 0.36 0.91 0.2  0.74 0.83 0.29 0.7
 0.18 0.15 0.79 0.66 0.17 0.85 0.68 0.33 0.31 0.11 0.4  0.32 0.1  0.63
 0.76 0.64 0.87 0.45 0.41 0.59 0.39 0.03 0.56 0.21 0.08 0.69 0.22 0.47
 0.58 0.72 0.37 0.55 0.28 0.52 0.06 0.48 0.09 0.34 0.13 0.35 0.54 0.51
 0.02 0.26 0.14 0.46 0.24 0.19 0.23 0.05 0.49 0.27 0.38 0.04 0.07 0.42]


In [227]:

df['host_acceptance_rate_binary'] = df['host_acceptance_rate'].apply(lambda x: 1 if x >= 0.8 else 0)


In [228]:

print(df['host_acceptance_rate_binary'].value_counts(normalize=True))


host_acceptance_rate_binary
1    0.872846
0    0.127154
Name: proportion, dtype: float64


In [229]:

#Declaramos las variables dependientes e independientes para la regresion lodistica
var_ind3 = df[['host_response_rate', 'calculated_host_listings_count', 'review_scores_communication']]
var_dep3 = df['host_acceptance_rate_binary']


In [230]:

#Redifinimos las variables
X = var_ind3
y = var_dep3


In [231]:

#Dividimos el conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)


In [232]:

#Se escalan todos los datos
escalar3 = StandardScaler()


In [233]:

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, se utiliza el método fit_transform
X_train3 = escalar3.fit_transform(X_train)
X_test3 = escalar3.transform(X_test)


In [234]:

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo3 = LogisticRegression()


In [235]:

#Entrenamos el algoritmo
algoritmo3.fit(X_train, y_train)


LogisticRegression()

In [236]:

y_pred3 = algoritmo3.predict(X_test)
y_pred3


array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [237]:

#Verificamos la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz3 = confusion_matrix(y_test, y_pred3)
print('Matriz de confusión:')
print(matriz3)


Matriz de confusión:
[[ 241  769]
 [  98 6867]]


In [238]:

#Verificamos la precisión
from sklearn.metrics import precision_score
precision3 = precision_score(y_test, y_pred3, average="binary", pos_label=1)
print('Precisión: del modelo', precision3)


Precisión: del modelo 0.8992928234677842


In [239]:

#calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud3 = accuracy_score(y_test, y_pred3)
print('Exactitud del modelo:', exactitud3)


Exactitud del modelo: 0.8912852664576802


In [240]:

#Calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad3 = recall_score(y_test, y_pred3, average="binary", pos_label=1)
print('Sensibilidad del modelo:', sensibilidad3)

Sensibilidad del modelo: 0.985929648241206


**Caso 4: room_type**

In [241]:
unico = np.unique(df['property_type'])
unico

array(['Boat', 'Campsite', 'Casa particular', 'Castle', 'Dome',
       'Earthen home', 'Entire bungalow', 'Entire cabin', 'Entire chalet',
       'Entire condo', 'Entire cottage', 'Entire guest suite',
       'Entire guesthouse', 'Entire home', 'Entire home/apt',
       'Entire hostel', 'Entire in-law', 'Entire loft', 'Entire place',
       'Entire rental unit', 'Entire serviced apartment',
       'Entire townhouse', 'Entire vacation home', 'Entire villa',
       'Farm stay', 'Holiday park', 'Hut', 'Private room',
       'Private room in barn', 'Private room in bed and breakfast',
       'Private room in bungalow', 'Private room in cabin',
       'Private room in casa particular', 'Private room in castle',
       'Private room in chalet', 'Private room in condo',
       'Private room in cottage', 'Private room in dome',
       'Private room in dorm', 'Private room in earthen home',
       'Private room in farm stay', 'Private room in floor',
       'Private room in guest suite', 'Priva

In [242]:
#Verificamos los datos unicos
df['property_type'].value_counts()


property_type
Entire rental unit             9971
Entire condo                   3423
Private room in rental unit    3034
Private room in home           2266
Entire loft                    1515
                               ... 
Private room in floor             1
Earthen home                      1
Entire in-law                     1
Shared room in dorm               1
Tower                             1
Name: count, Length: 92, dtype: int64

In [243]:
#Volvemos la variable dinatomica 
df['property_type'] = np.where(df['property_type'] == 'Entire rental unit', 'Entire rental unit', 'Other property')

In [244]:
unico = np.unique(df['property_type'])
unico

array(['Entire rental unit', 'Other property'], dtype=object)

In [245]:
room_unico = np.unique(df['room_type'])
room_unico

array(['Entire home/apt', 'Hotel room', 'Private room', 'Shared room'],
      dtype=object)

In [246]:
#Mapeamos los valores de la columna 'property_type' a valores numericos
mapeo_room = {tipo: i + 1 for i, tipo in enumerate(room_unico)}

In [247]:
# Aplicamos el mapeo a la columna 'property_type'
df['room_type'] = df['room_type'].map(mapeo_room)
df['room_type'].head(10)

0    1
1    1
2    1
3    1
4    1
5    3
6    1
7    3
8    1
9    1
Name: room_type, dtype: int64

In [248]:
#Declaramos las variables dependientes e independientes para la regresion lodistica
var_ind4 = df[['room_type', 'review_scores_cleanliness', 'review_scores_communication']]
var_dep4 = df['property_type']

In [249]:
#Redifinimos las variables
X = var_ind4
y = var_dep4

In [250]:
#Dividimos el conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [251]:
#Se escalan todos los datos
escalar4 = StandardScaler()

In [252]:
#Para realizar el escalamiento de las variables "X" tanto en entrenamiento como en prueba
X_train = escalar4.fit_transform(X_train)
X_test = escalar4.transform(X_test) 

In [253]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo4 = LogisticRegression()

In [254]:
#Entrenamos el algoritmo
algoritmo4.fit(X_train, y_train)

LogisticRegression()

In [255]:
y_pred4 = algoritmo4.predict(X_test)
y_pred4

array(['Other property', 'Entire rental unit', 'Other property', ...,
       'Entire rental unit', 'Entire rental unit', 'Entire rental unit'],
      dtype=object)

In [256]:
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred4)
print('Matriz de confusión:')
print(matriz)

Matriz de confusión:
[[3009    0]
 [2198 2768]]


In [257]:
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred4, average="binary", pos_label="Entire rental unit")
print('Precisión del modelo')
print(precision)

Precisión del modelo
0.5778759362396774


In [258]:
#calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred4)
print('Exactitud del modelo:', exactitud)

Exactitud del modelo: 0.7243887147335423


In [259]:
#Calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred4, average="binary", pos_label="Other property")
print('Sensibilidad del modelo:', sensibilidad)

Sensibilidad del modelo: 0.5573902537253322


**Caso 5: review_scores_cleanliness**

In [260]:
df['review_scores_cleanliness'].head()

0    4.84
1    4.70
2    4.75
3    4.96
4    5.00
Name: review_scores_cleanliness, dtype: float64

In [261]:
unico = np.unique(df['review_scores_cleanliness'])
unico

array([4.43, 4.44, 4.45, 4.46, 4.47, 4.48, 4.49, 4.5 , 4.51, 4.52, 4.53,
       4.54, 4.55, 4.56, 4.57, 4.58, 4.59, 4.6 , 4.61, 4.62, 4.63, 4.64,
       4.65, 4.66, 4.67, 4.68, 4.69, 4.7 , 4.71, 4.72, 4.73, 4.74, 4.75,
       4.76, 4.77, 4.78, 4.79, 4.8 , 4.81, 4.82, 4.83, 4.84, 4.85, 4.86,
       4.87, 4.88, 4.89, 4.9 , 4.91, 4.92, 4.93, 4.94, 4.95, 4.96, 4.97,
       4.98, 4.99, 5.  ])

In [262]:
#Convierto la variable a dicotomica
df['review_scores_cleanliness'] = (df['review_scores_cleanliness'] > 4.5 ).astype(int)
df['review_scores_cleanliness'].head(10)

0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
Name: review_scores_cleanliness, dtype: int32

In [263]:
type_unico = np.unique( df['property_type'])
type_unico

array(['Entire rental unit', 'Other property'], dtype=object)

In [264]:
mapeo_type = {tipo: i + 1 for i, tipo in enumerate(type_unico)}

In [265]:
# Aplicamos el mapeo a la columna 'property_type'
df['property_type'] = df['property_type'].map(mapeo_type)
df['property_type'].head(10)

0    2
1    2
2    1
3    1
4    2
5    2
6    2
7    2
8    2
9    2
Name: property_type, dtype: int64

In [266]:
#Declaramos las variables dependientes e independientes para la regresion lodistica
var_ind5 = df[['property_type', 'room_type', 'number_of_reviews']]
var_dep5 = df['review_scores_cleanliness']

In [267]:
#Redifinimos las variables
X = var_ind5
y = var_dep5

In [268]:
#Dividimos el conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [269]:
#Se escalan todos los datos
escalar5 = StandardScaler()

In [270]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, se utiliza el método fit_transform
X_train = escalar5.fit_transform(X_train)
X_test = escalar5.transform(X_test)


In [271]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo5 = LogisticRegression()


In [272]:

#Entrenamos el algoritmo
algoritmo5.fit(X_train, y_train)


LogisticRegression()

In [273]:

#Realizamos la prediccion
y_pred5 = algoritmo5.predict(X_test)
y_pred5


array([1, 1, 1, ..., 1, 1, 1])

In [274]:

#Verificamos la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred5)
print('Matriz de confusión:')
print(matriz)


Matriz de confusión:
[[   0  255]
 [   0 7720]]


In [275]:

#Verificamos la precisión
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred5, average="binary", pos_label=1)
print('Precisión: del modelo', precision)


Precisión: del modelo 0.9680250783699059


In [276]:

#calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred5)
print('Exactitud del modelo:', exactitud)


Exactitud del modelo: 0.9680250783699059


In [277]:

#Calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred5, average="binary", pos_label=0)
print('Sensibilidad del modelo:', sensibilidad)


Sensibilidad del modelo: 0.0


**Caso 6: review_scores_communication**

In [278]:
df['host_verifications']

0        ['email', 'phone', 'work_email']
1        ['email', 'phone', 'work_email']
2                      ['email', 'phone']
3        ['email', 'phone', 'work_email']
4                      ['email', 'phone']
                       ...               
26577                  ['email', 'phone']
26578                  ['email', 'phone']
26579                  ['email', 'phone']
26580                  ['email', 'phone']
26581                  ['email', 'phone']
Name: host_verifications, Length: 26582, dtype: object

In [279]:
verifications_unico = np.unique(df['host_verifications'])
verifications_unico

array(["['email', 'phone', 'work_email']", "['email', 'phone']",
       "['email', 'work_email']", "['email']", "['phone', 'work_email']",
       "['phone']", "['work_email']", '[]'], dtype=object)

In [280]:
#Mapeamos los valores de la columna 'host_verificactions' a valores numericos
mapeo_verifications = {tipo: i + 1 for i, tipo in enumerate(verifications_unico)}

In [281]:
# Aplicamos el mapeo a la columna 'property_type'
df['host_verifications'] = df['host_verifications'].map(mapeo_verifications)
df['host_verifications'].head(10)

0    1
1    1
2    2
3    1
4    2
5    6
6    2
7    2
8    2
9    2
Name: host_verifications, dtype: int64

In [282]:
df['review_scores_communication']

0        4.80
1        4.78
2        4.92
3        4.98
4        4.73
         ... 
26577    4.80
26578    4.80
26579    4.80
26580    4.80
26581    4.80
Name: review_scores_communication, Length: 26582, dtype: float64

In [283]:
unico = np.unique(df['review_scores_communication'])
unico

array([4.5 , 4.51, 4.52, 4.53, 4.54, 4.55, 4.56, 4.57, 4.58, 4.59, 4.6 ,
       4.61, 4.62, 4.63, 4.64, 4.65, 4.66, 4.67, 4.68, 4.69, 4.7 , 4.71,
       4.72, 4.73, 4.74, 4.75, 4.76, 4.77, 4.78, 4.79, 4.8 , 4.81, 4.82,
       4.83, 4.84, 4.85, 4.86, 4.87, 4.88, 4.89, 4.9 , 4.91, 4.92, 4.93,
       4.94, 4.95, 4.96, 4.97, 4.98, 4.99, 5.  ])

In [284]:
#Convierto la variable a dicotomica
df['review_scores_communication'] = (df['review_scores_communication'] > 4.85 ).astype(int)

In [285]:
df['review_scores_communication'].head(10)

0    0
1    0
2    1
3    1
4    0
5    0
6    1
7    1
8    1
9    1
Name: review_scores_communication, dtype: int32

In [286]:
#identificamos valores unicos 
unico = np.unique(df['review_scores_value'])
unico

array([4.38, 4.39, 4.4 , 4.41, 4.42, 4.43, 4.44, 4.45, 4.46, 4.47, 4.48,
       4.49, 4.5 , 4.51, 4.52, 4.53, 4.54, 4.55, 4.56, 4.57, 4.58, 4.59,
       4.6 , 4.61, 4.62, 4.63, 4.64, 4.65, 4.66, 4.67, 4.68, 4.69, 4.7 ,
       4.71, 4.72, 4.73, 4.74, 4.75, 4.76, 4.77, 4.78, 4.79, 4.8 , 4.81,
       4.82, 4.83, 4.84, 4.85, 4.86, 4.87, 4.88, 4.89, 4.9 , 4.91, 4.92,
       4.93, 4.94, 4.95, 4.96, 4.97, 4.98, 4.99, 5.  ])

In [287]:
#Convierto la variable a dicotomica
df['review_scores_value'] = (df['review_scores_value'] > 4.5 ).astype(int)
df['review_scores_value'].head(10)

0    1
1    0
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
Name: review_scores_value, dtype: int32

In [288]:
df['number_of_reviews']

0          0.0
1         64.0
2         49.0
3        127.0
4         11.0
         ...  
26577      0.0
26578      0.0
26579      0.0
26580      0.0
26581      0.0
Name: number_of_reviews, Length: 26582, dtype: float64

In [289]:
#Declaramos las variables dependientes e independientes para la regresion lodistica
var_ind6 = df[['number_of_reviews', 'reviews_per_month', 'host_verifications']]
var_dep6 = df['review_scores_communication']


In [290]:

#Redifinimos las variables
X = var_ind6
y = var_dep6


In [291]:

#Dividimos el conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)


In [292]:

#Se escalan todos los datos
escalar6 = StandardScaler()


In [293]:

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, se utiliza el método fit_transform
X_train = escalar6.fit_transform(X_train)
X_test = escalar6.transform(X_test)


In [294]:

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo6 = LogisticRegression()


In [295]:
#Entrenamos el algoritmo
algoritmo6.fit(X_train, y_train)


LogisticRegression()

In [296]:

#Realizamos la prediccion
y_pred6 = algoritmo6.predict(X_test)
y_pred6


array([0, 1, 1, ..., 1, 1, 1])

In [297]:

#Verificamos la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred6)
print('Matriz de confusión:')
print(matriz)


Matriz de confusión:
[[ 293 2491]
 [ 315 4876]]


In [298]:

#Verificamos la precisión
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred6, average="binary", pos_label=1)
print('Precisión: del modelo', precision)


Precisión: del modelo 0.6618705035971223


In [299]:

#calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred6)
print('Exactitud del modelo:', exactitud)


Exactitud del modelo: 0.6481504702194357


In [300]:

#Calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred6, average="binary", pos_label=0)
print('Sensibilidad del modelo:', sensibilidad)



Sensibilidad del modelo: 0.10524425287356322


**Caso 7: calculated_host_listings_count**

In [301]:
#Verificamos los datos unicos de la columna 'class'
unico = np.unique(df['calculated_host_listings_count'])
unico

array([ 1. ,  2. ,  3. ,  4. ,  5. ,  6. ,  7. ,  8. ,  9. ,  9.8, 10. ,
       11. , 12. , 13. , 14. , 15. , 16. , 17. , 18. , 19. , 20. , 21. ,
       22. , 23. , 24. , 25. , 26. ])

In [302]:
#Convierto la variable a dicotomica
df['calculated_host_listings_count'] = (df['calculated_host_listings_count'] > 9.8 ).astype(int)
df['calculated_host_listings_count'].head(10)

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: calculated_host_listings_count, dtype: int32

In [303]:
#Declaramos las variables dependientes e independientes para la regresion lodistica
var_ind7 = df[['host_is_superhost', 'host_response_rate', 'host_acceptance_rate']]
var_dep7 = df[['calculated_host_listings_count']]

In [304]:
#Redifinimos las variables
X = var_ind7
y = var_dep7

In [305]:
#Dividimos el conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [306]:
#Se escalan todos los datos
escalar7 = StandardScaler()

In [307]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, se utiliza el método fit_transform
X_train = escalar7.fit_transform(X_train)
X_test = escalar7.transform(X_test)

In [308]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo7 = LogisticRegression()

In [309]:
#Entrenamos el algoritmo
algoritmo7.fit(X_train, y_train)

c:\Users\Bonil\OneDrive\Escritorio\Archivos Buap\Inteligencia de Negocios\env\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [310]:
#Realizamos la prediccion
y_pred7 = algoritmo7.predict(X_test)
y_pred7

array([0, 0, 0, ..., 0, 0, 0])

In [311]:
#Verificamos la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred7)
print('Matriz de confusión:')
print(matriz)

Matriz de confusión:
[[6876    0]
 [1099    0]]


In [312]:
#Verificamos la precisión
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred7, average="binary", pos_label=1)
print('Precisión: del modelo', precision)

Precisión: del modelo 0.0


c:\Users\Bonil\OneDrive\Escritorio\Archivos Buap\Inteligencia de Negocios\env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [313]:
#calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred7)
print('Exactitud del modelo:', exactitud)

Exactitud del modelo: 0.8621943573667712


In [314]:
#Calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred7, average="binary", pos_label=0)
print('Sensibilidad del modelo:', sensibilidad)

Sensibilidad del modelo: 1.0


**Caso 8: reviews_per_month**

In [315]:
#Verificamos los datos unicos de la columna
unico = np.unique(df['reviews_per_month'])
unico

array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99,
       1.  , 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07, 1.08, 1.09, 1.1 ,
       1.11, 1.12, 1.13, 1.14, 1.15, 1.16, 1.17, 1.18, 1.19, 1.2 , 1.21,
       1.22, 1.23, 1.24, 1.25, 1.26, 1.27, 1.28, 1.29, 1.3 , 1.31, 1.32,
       1.33, 1.34, 1.35, 1.36, 1.37, 1.38, 1.39, 1.4 , 1.41, 1.42, 1.43,
       1.44, 1.45, 1.46, 1.47, 1.48, 1.49, 1.5 , 1.

In [316]:
#Convierto la variable a dicotomica
df['reviews_per_month'] = (df['reviews_per_month'] > 2.0 ).astype(int)
df['reviews_per_month'].head(10)

0    0
1    0
2    0
3    0
4    0
5    0
6    1
7    0
8    0
9    0
Name: reviews_per_month, dtype: int32

In [317]:
#Declaramos las variables dependientes e independientes para la regresion logistica
var_ind8 = df[['host_is_superhost', 'host_response_rate', 'host_acceptance_rate']]
var_dep8 = df[['reviews_per_month']]

In [318]:
#Redefinimos las variables
X = var_ind8
y = var_dep8

In [319]:
#Dividimos el conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [320]:
#Se escalan todos los datos
escalar8 = StandardScaler()

In [321]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, se utiliza el método fit_transform
X_train = escalar8.fit_transform(X_train)
X_test = escalar8.transform(X_test)

In [322]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo8 = LogisticRegression()

In [323]:
#Entrenamos el algoritmo
algoritmo8.fit(X_train, y_train)

c:\Users\Bonil\OneDrive\Escritorio\Archivos Buap\Inteligencia de Negocios\env\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [324]:
#Realizamos la prediccion
y_pred8 = algoritmo8.predict(X_test)
y_pred8

array([0, 0, 0, ..., 0, 0, 0])

In [325]:
#Verificamos la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred8)
print('Matriz de confusión:')
print(matriz)

Matriz de confusión:
[[6194    0]
 [1781    0]]


In [326]:
#Verificamos la precisión
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred8, average="binary", pos_label=1)
print('Precisión: del modelo', precision)

Precisión: del modelo 0.0


c:\Users\Bonil\OneDrive\Escritorio\Archivos Buap\Inteligencia de Negocios\env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [327]:
#calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred8)
print('Exactitud del modelo:', exactitud)

Exactitud del modelo: 0.7766771159874608


In [328]:
#Calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred8, average="binary", pos_label=0)
print('Sensibilidad del modelo:', sensibilidad)

Sensibilidad del modelo: 1.0


**Caso 9: neighbourhood_cleansed**

In [329]:
#Verificamos los datos unicos de la columna 'property_type'
property_unico = np.unique(df['property_type'])
property_unico 


array([1, 2], dtype=int64)

In [330]:
#Cantidad de variables
num_property_unico = len(property_unico)
print(f"La cantidad de variables únicas en 'property_type' es: {num_property_unico}")



La cantidad de variables únicas en 'property_type' es: 2


In [331]:
#Mapeamos los valores de la columna 'property_type' a valores numericos
mapeo_property = {tipo: i for i, tipo in enumerate(property_unico)}

In [332]:
# Aplicamos el mapeo a la columna 'property_type'
df['property_type'] = df['property_type'].map(mapeo_property)
df['property_type'].head(10)

0    1
1    1
2    0
3    0
4    1
5    1
6    1
7    1
8    1
9    1
Name: property_type, dtype: int64

In [333]:
#Verificamos los datos unicos de la columna 
unico = np.unique(df['neighbourhood_cleansed'])
unico

array(['Azcapotzalco', 'Benito Juárez', 'Coyoacán',
       'Cuajimalpa de Morelos', 'Cuauhtémoc', 'Gustavo A. Madero',
       'Iztacalco', 'Iztapalapa', 'La Magdalena Contreras',
       'Miguel Hidalgo', 'Milpa Alta', 'Tlalpan', 'Tláhuac',
       'Venustiano Carranza', 'Xochimilco', 'Álvaro Obregón'],
      dtype=object)

In [334]:
#Seleccionamos 5 zonas al azar para volver a la variable dicotomica
zonas = [
    'Iztapalapa', 'Xochimilco', 'Tlalpan', 
    'Azcapotzalco', 'Coyoacán'
]

In [335]:
#Convertimos la variable a dicotomica
df['neighbourhood_cleansed'] = df['neighbourhood_cleansed'].isin(zonas).astype(int)
df['neighbourhood_cleansed'].head(10)

0    0
1    0
2    0
3    1
4    0
5    1
6    0
7    0
8    1
9    0
Name: neighbourhood_cleansed, dtype: int32

In [336]:
#Declaramos las variables dependientes e independientes para la regresion logistica
var_ind9 = df[['property_type', 'minimum_nights', 'review_scores_location']]
var_dep9 = df[['neighbourhood_cleansed']]

In [337]:
#Redifinimos las variables
X = var_ind9
y = var_dep9


In [338]:

#Dividimos el conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)


In [339]:

#Se escalan todos los datos
escalar9 = StandardScaler()


In [340]:

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, se utiliza el método fit_transform
X_train = escalar9.fit_transform(X_train)
X_test = escalar9.transform(X_test)


In [341]:

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo9 = LogisticRegression()


In [342]:

#Entrenamos el algoritmo
algoritmo9.fit(X_train, y_train)


c:\Users\Bonil\OneDrive\Escritorio\Archivos Buap\Inteligencia de Negocios\env\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [343]:

#Realizamos la prediccion
y_pred9 = algoritmo9.predict(X_test)
y_pred9


array([0, 0, 0, ..., 0, 0, 0])

In [344]:

#Verificamos la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred9)
print('Matriz de confusión:')
print(matriz)


Matriz de confusión:
[[6975    0]
 [1000    0]]


In [345]:

#Verificamos la precisión
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred9, average="binary", pos_label=1)
print('Precisión: del modelo', precision)


Precisión: del modelo 0.0


c:\Users\Bonil\OneDrive\Escritorio\Archivos Buap\Inteligencia de Negocios\env\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [346]:

#calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred9)
print('Exactitud del modelo:', exactitud)


Exactitud del modelo: 0.8746081504702194


In [347]:

#Calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred9, average="binary", pos_label=0)
print('Sensibilidad del modelo:', sensibilidad)


Sensibilidad del modelo: 1.0


**Caso 10: host_response_time**

In [348]:
host_unico = np.unique(df['host_response_time'])
host_unico

array(['a few days or more', 'within a day', 'within a few hours',
       'within an hour'], dtype=object)

In [349]:
#volver a convertir la variable a dicotomica
df['host_response_time'] = df['host_response_time'].replace(['a few days or more','within a few hours','within a day'], 'Respond at another time')

In [350]:
#Declaramos las variables dependientes e independientes para la regresion logistica
var_ind10 = df[['host_response_rate', 'review_scores_communication', 'host_acceptance_rate']]
var_dep10 = df[['host_response_time']]

In [351]:
#Redifinimos las variables
X = var_ind10
y = var_dep10


In [352]:

#Dividimos el conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)


In [353]:

#Se escalan todos los datos
escalar10 = StandardScaler()


In [354]:

#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, se utiliza el método fit_transform
X_train = escalar10.fit_transform(X_train)
X_test = escalar10.transform(X_test)


In [355]:

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo10 = LogisticRegression()


In [356]:

#Entrenamos el algoritmo
algoritmo10.fit(X_train, y_train)


c:\Users\Bonil\OneDrive\Escritorio\Archivos Buap\Inteligencia de Negocios\env\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [357]:

#Realizamos la prediccion
y_pred10 = algoritmo10.predict(X_test)
y_pred10


array(['within an hour', 'within an hour', 'Respond at another time', ...,
       'within an hour', 'within an hour', 'Respond at another time'],
      dtype=object)

In [358]:

#Verificamos la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred10)
print('Matriz de confusión:')
print(matriz)


Matriz de confusión:
[[ 907 1484]
 [ 204 5380]]


In [359]:

#Verificamos la precisión
from sklearn.metrics import precision_score
precision = precision_score(y_test, y_pred10, average="binary", pos_label="within an hour")
print('Precisión: del modelo', precision)


Precisión: del modelo 0.7837995337995338


In [360]:

#calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(y_test, y_pred10)
print('Exactitud del modelo:', exactitud)


Exactitud del modelo: 0.7883385579937304


In [361]:

#Calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred10, average="binary", pos_label="Respond at another time")
print('Sensibilidad del modelo:', sensibilidad)


Sensibilidad del modelo: 0.3793391886240067
